# Init hosted instance

In [ ]:
!lamin login testuser1

In [ ]:
import lamindb_setup as ln_setup
from lamindb_setup.core.upath import create_path, UPath, InstanceNotEmpty
from lamindb_setup.core._hub_core import (
    delete_instance,
    call_with_fallback_auth,
    select_instance_by_owner_name,
)
import pytest

instance_name = "my-hosted"
assert ln_setup.settings.user.handle == "testuser1"

In [ ]:
try:
    delete_instance(f"testuser1/{instance_name}")
except InstanceNotEmpty:
    instance_with_storage = call_with_fallback_auth(
        select_instance_by_owner_name,
        owner="testuser1",
        name=instance_name,
    )
    root = create_path(instance_with_storage["storage"]["root"])
    for obj in root.rglob(""):
        if obj.is_file():
            obj.unlink()
    delete_instance(f"testuser1/{instance_name}")

In [ ]:
with pytest.raises(ValueError):
    ln_setup.init(storage="create-s3")

In [ ]:
ln_setup.init(name="my-hosted", storage="create-s3")

In [ ]:
from lamindb_setup.core._aws_credentials import HOSTED_BUCKETS

In [ ]:
assert ln_setup.settings.instance.storage.type_is_cloud == True
assert ln_setup.settings.instance.owner == ln_setup.settings.user.handle
assert ln_setup.settings.instance.name == "my-hosted"
assert ln_setup.settings.storage.root.as_posix().startswith(HOSTED_BUCKETS)
assert ln_setup.settings.storage.id is not None

assert ln_setup.settings.storage._mark_storage_root.exists()

In [ ]:
ln_setup.close()